In [1]:
%reset
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import defmod as dm

torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# WIP
# On va merge les deux .pkl en un seul dictionnary
data_source = pickle.load(open("../data/basi1b.pkl", 'rb'))
data_target = pickle.load(open("../data/basi1t.pkl", 'rb'))

Dx = 0.
Dy = 0.
height_source = 38.
height_target = 100.

source = torch.tensor(data_source[1]).type(torch.get_default_dtype())
target = torch.tensor(data_target[1]).type(torch.get_default_dtype())

smin, smax = torch.min(source[:, 1]), torch.max(source[:, 1])
sscale = height_source / (smax - smin)
source[:, 1] = Dy - sscale * (source[:, 1] - smax)
source[:, 0] = Dx + sscale * (source[:, 0] - torch.mean(source[:, 0]))

tmin, tmax = torch.min(target[:, 1]), torch.max(target[:, 1])
tscale = height_target / (tmax - tmin)
target[:, 1] = - tscale * (target[:, 1] - tmax)
target[:, 0] = tscale * (target[:, 0] - torch.mean(target[:, 0]))

pos_source = source[source[:, 2] == 2, 0:2]
pos_source = torch.tensor(np.delete(pos_source.numpy(), 3, axis=0))
pos_implicit0 = source[source[:, 2] == 1, 0:2]
pos_implicit1 = source[source[:, 2] == 1, 0:2]
pos_target = target[target[:, 2] == 2, 0:2]

aabb = dm.usefulfunctions.AABB.build_from_points(pos_target)
aabb.squared()

In [3]:
# Some plots
%matplotlib qt5

plt.subplot(2, 2, 1)
plt.axis(aabb.get_list())
plt.title('Source')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')

plt.subplot(2, 2, 2)
plt.axis(aabb.get_list())
plt.title('Target')
plt.xlabel('x')
plt.ylabel('y')
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')

plt.subplot(2, 2, 3)
plt.imshow(data_source[0])

plt.subplot(2, 2, 4)
plt.imshow(data_target[0])

plt.show()

In [4]:
# Setting up the modules
# Local translation module
sigma0 = 15.
nu0 = 0.001
coeff0 = 100.
implicit0 = dm.implicitmodules.ImplicitModule0(dm.manifold.Landmarks(2, pos_implicit0.shape[0], gd=pos_implicit0.view(-1).requires_grad_()), sigma0, nu0, coeff0)

# Global translation module
sigma00 = 400.
nu00 = 0.001
coeff00 = 0.01
implicit00 = dm.implicitmodules.ImplicitModule0(dm.manifold.Landmarks(2, 1, gd=torch.tensor([0., 0.], requires_grad=True)), sigma00, nu00, coeff00)

# Elastic modules
sigma1 = 30.
nu1 = 0.001
coeff1 = 0.01
C = torch.zeros(pos_implicit1.shape[0], 2, 1)
K, L = 10, height_source
a, b = -2 / L ** 3, 3 / L ** 2
C[:, 1, 0] = (K * (a * (L - pos_implicit1[:, 1] + Dy) ** 3  + b * (L - pos_implicit1[:, 1] + Dy) ** 2))
C[:, 0, 0] = 1. * C[:, 1, 0]
th = 0. * math.pi * torch.ones(pos_implicit1.shape[0])
R = torch.stack([dm.usefulfunctions.rot2d(t) for t in th])

implicit1 = dm.implicitmodules.ImplicitModule1(dm.manifold.Stiefel(2, pos_implicit1.shape[0], gd=(pos_implicit1.view(-1).requires_grad_(), R.view(-1).requires_grad_())), C, sigma1, nu1, coeff1)

In [5]:
# Setting up the model and start the fitting loop
model = dm.models.ModelCompoundWithPointsRegistration((pos_source, torch.ones(pos_source.shape[0])), [implicit0, implicit00, implicit1], [True, True, True], dm.attachement.VarifoldAttachement([10., 50.]))
costs = model.fit((pos_target, torch.ones(pos_target.shape[0])), max_iter=40, l=1., lr=5e-1, log_interval=1)

___________
hamiltonian
[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], requires_grad=True), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True), tensor([0., 0.], requires_grad=True), (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

___________
hamiltonian
[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], grad_fn=<AddBackward0>), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<AddBackward0>), tensor([0., 0.], grad_fn=<AddBackward0>), (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

[tensor([-1.7849e-04,  5.0610e-03, -1.9293e-04,  5.0530e-03, -2.1310e-04,
         5.0376e-03, -2.4308e-04,  5.0087e-03, -2.7321e-04,  4.9652e-03,
        -3.0356e-04,  4.9146e-03, -3.3210e-04,  4.8512e-03, -3.6027e-04,
         4.7773e-03, -3.8780e-04,  4.6910e-03, -4.0206e-04,  4.5942e-03,
        -4.2069e-04,  4.5016e-03, -4.2389e-04,  4.3949e-03, -4.1729e-04,
         4.2872e-03, -4.0253e-04,  4.1882e-03, -3.7651e-04,  4.1101e-03,
        -3.3582e-04,  4.0203e-03, -2.9183e-04,  3.9220e-03, -2.3404e-04,
         3.8722e-03, -1.9829e-04,  3.7915e-03, -1.8512e-04,  3.7347e-03,
        -1.1071e-04,  3.7216e-03, -4.9970e-05,  3.7238e-03, -6.7399e-05,
         3.7822e-03, -5.8833e-05,  3.8684e-03,  9.1885e-06,  3.9728e-03,
         5.3067e-05,  4.1169e-03,  8.1445e-05,  4.2469e-03,  8.1986e-05,
         4.4018e-03,  7.4919e-05,  4.5248e-03,  5.8675e-05,  4.6296e-03,
         4.2326e-05,  4.7289e-03,  1.7247e-05,  4.8165e-03, -1.5441e-05,
         4.8848e-03, -4.9464e-05,  4.9405e-03, -8.

___________
hamiltonian
[tensor([-1.7881e-04,  5.1159e-03, -1.9489e-04,  5.0903e-03, -2.1585e-04,
         5.0595e-03, -2.4584e-04,  5.0109e-03, -2.7459e-04,  4.9449e-03,
        -3.0299e-04,  4.8735e-03, -3.2870e-04,  4.7884e-03, -3.5346e-04,
         4.6936e-03, -3.7692e-04,  4.5870e-03, -3.8699e-04,  4.4713e-03,
        -4.0155e-04,  4.3643e-03, -4.0102e-04,  4.2444e-03, -3.9161e-04,
         4.1269e-03, -3.7533e-04,  4.0218e-03, -3.4965e-04,  3.9406e-03,
        -3.1097e-04,  3.8495e-03, -2.6998e-04,  3.7527e-03, -2.1785e-04,
         3.7044e-03, -1.8559e-04,  3.6288e-03, -1.7373e-04,  3.5772e-03,
        -1.0814e-04,  3.5653e-03, -5.4639e-05,  3.5676e-03, -6.9830e-05,
         3.6203e-03, -6.1989e-05,  3.7004e-03, -6.6997e-07,  3.8012e-03,
         4.0377e-05,  3.9454e-03,  6.8177e-05,  4.0810e-03,  7.1350e-05,
         4.2485e-03,  6.7165e-05,  4.3873e-03,  5.3988e-05,  4.5097e-03,
         4.0681e-05,  4.6303e-03,  1.8548e-05,  4.7409e-03, -1.1956e-05,
         4.8308e-03, -4.45

___________
hamiltonian
[tensor([-1.1460e-04,  4.9405e-03, -1.3522e-04,  4.9342e-03, -1.6457e-04,
         4.9167e-03, -2.0858e-04,  4.8812e-03, -2.5328e-04,  4.8253e-03,
        -2.9840e-04,  4.7587e-03, -3.4111e-04,  4.6740e-03, -3.8344e-04,
         4.5742e-03, -4.2499e-04,  4.4563e-03, -4.4723e-04,  4.3231e-03,
        -4.7580e-04,  4.1945e-03, -4.8183e-04,  4.0454e-03, -4.7334e-04,
         3.8938e-03, -4.5260e-04,  3.7534e-03, -4.1482e-04,  3.6421e-03,
        -3.5523e-04,  3.5134e-03, -2.9055e-04,  3.3712e-03, -2.0470e-04,
         3.2988e-03, -1.5166e-04,  3.1804e-03, -1.3216e-04,  3.0963e-03,
        -2.0665e-05,  3.0769e-03,  7.0424e-05,  3.0801e-03,  4.4176e-05,
         3.1669e-03,  5.6871e-05,  3.2937e-03,  1.5820e-04,  3.4457e-03,
         2.2313e-04,  3.6535e-03,  2.6489e-04,  3.8391e-03,  2.6525e-04,
         4.0578e-03,  2.5451e-04,  4.2299e-03,  2.3033e-04,  4.3752e-03,
         2.0604e-04,  4.5117e-03,  1.6900e-04,  4.6308e-03,  1.2095e-04,
         4.7224e-03,  7.11

...

It: 38, deformation cost: 50.486044, attach: 514.792546. Total cost: 565.278590
___________
hamiltonian
[tensor([-0.0014,  0.0762, -0.0040,  0.0785, -0.0080,  0.0789, -0.0143,  0.0776,
        -0.0211,  0.0739, -0.0280,  0.0683, -0.0347,  0.0602, -0.0415,  0.0498,
        -0.0483,  0.0366, -0.0523,  0.0210, -0.0572,  0.0051, -0.0588, -0.0140,
        -0.0580, -0.0341, -0.0552, -0.0533, -0.0496, -0.0688, -0.0406, -0.0871,
        -0.0306, -0.1080, -0.0171, -0.1187, -0.0087, -0.1369, -0.0056, -0.1501,
         0.0122, -0.1532,  0.0267, -0.1527,  0.0225, -0.1390,  0.0245, -0.1194,
         0.0404, -0.0968,  0.0503, -0.0668,  0.0565, -0.0410,  0.0561, -0.0118,
         0.0540,  0.0101,  0.0498,  0.0277,  0.0456,  0.0434,  0.0396,  0.0562,
         0.0322,  0.0653,  0.0246,  0.0718,  0.0163,  0.0764,  0.0102,  0.0784,
         0.0040,  0.0791,  0.0004,  0.0782], requires_grad=True), tensor([-0.0037,  0.0789,  0.0004,  0.0788, -0.0129,  0.0772, -0.0053,  0.0778,
         0.0023,  0.0781,  0.

___________
hamiltonian
[tensor([-0.0013,  0.0846, -0.0037,  0.0826, -0.0072,  0.0794, -0.0120,  0.0741,
        -0.0164,  0.0664, -0.0203,  0.0585, -0.0234,  0.0492, -0.0261,  0.0395,
        -0.0284,  0.0294, -0.0286,  0.0188, -0.0298,  0.0104, -0.0292,  0.0009,
        -0.0279, -0.0080, -0.0261, -0.0158, -0.0234, -0.0221, -0.0193, -0.0294,
        -0.0153, -0.0383, -0.0090, -0.0432, -0.0054, -0.0530, -0.0041, -0.0615,
         0.0061, -0.0637,  0.0146, -0.0637,  0.0114, -0.0544,  0.0116, -0.0435,
         0.0188, -0.0331,  0.0228, -0.0211,  0.0261, -0.0109,  0.0270,  0.0013,
         0.0277,  0.0123,  0.0274,  0.0228,  0.0273,  0.0346,  0.0257,  0.0461,
         0.0224,  0.0555,  0.0184,  0.0637,  0.0131,  0.0709,  0.0086,  0.0755,
         0.0037,  0.0804,  0.0005,  0.0834], grad_fn=<AddBackward0>), tensor([-0.0034,  0.0817,  0.0005,  0.0820, -0.0106,  0.0720, -0.0043,  0.0719,
         0.0019,  0.0729,  0.0075,  0.0742, -0.0174,  0.0591, -0.0048,  0.0569,
         0.0030,  0.0572,

___________
hamiltonian
[tensor([-0.0009,  0.0885, -0.0034,  0.0840, -0.0065,  0.0790, -0.0106,  0.0718,
        -0.0139,  0.0626, -0.0167,  0.0539, -0.0186,  0.0443, -0.0200,  0.0352,
        -0.0211,  0.0265, -0.0205,  0.0175, -0.0211,  0.0112, -0.0203,  0.0042,
        -0.0193, -0.0020, -0.0183, -0.0072, -0.0166, -0.0114, -0.0142, -0.0162,
        -0.0122, -0.0226, -0.0077, -0.0263, -0.0055, -0.0358, -0.0049, -0.0453,
         0.0050, -0.0478,  0.0135, -0.0483,  0.0096, -0.0373,  0.0088, -0.0263,
         0.0134, -0.0183,  0.0153, -0.0104,  0.0174, -0.0039,  0.0182,  0.0043,
         0.0192,  0.0124,  0.0197,  0.0207,  0.0206,  0.0311,  0.0204,  0.0420,
         0.0185,  0.0511,  0.0158,  0.0598,  0.0117,  0.0679,  0.0081,  0.0736,
         0.0038,  0.0802,  0.0009,  0.0853], grad_fn=<AddBackward0>), tensor([-2.9600e-03,  8.2401e-02,  7.7973e-04,  8.2913e-02, -9.1714e-03,
         6.8936e-02, -3.6303e-03,  6.8509e-02,  1.8993e-03,  6.9705e-02,
         6.9715e-03,  7.1699e-02, -1.42

___________
hamiltonian
[tensor([-0.0013,  0.0846, -0.0037,  0.0826, -0.0072,  0.0794, -0.0120,  0.0741,
        -0.0164,  0.0664, -0.0203,  0.0585, -0.0234,  0.0492, -0.0261,  0.0395,
        -0.0284,  0.0294, -0.0286,  0.0188, -0.0298,  0.0104, -0.0292,  0.0009,
        -0.0279, -0.0080, -0.0261, -0.0158, -0.0234, -0.0221, -0.0193, -0.0294,
        -0.0153, -0.0383, -0.0090, -0.0432, -0.0054, -0.0530, -0.0041, -0.0615,
         0.0061, -0.0637,  0.0146, -0.0637,  0.0114, -0.0544,  0.0116, -0.0435,
         0.0188, -0.0331,  0.0228, -0.0211,  0.0261, -0.0109,  0.0270,  0.0013,
         0.0277,  0.0123,  0.0274,  0.0228,  0.0273,  0.0346,  0.0257,  0.0461,
         0.0224,  0.0555,  0.0184,  0.0637,  0.0131,  0.0709,  0.0086,  0.0755,
         0.0037,  0.0803,  0.0005,  0.0833], grad_fn=<AddBackward0>), tensor([-0.0034,  0.0817,  0.0005,  0.0820, -0.0106,  0.0720, -0.0043,  0.0719,
         0.0019,  0.0729,  0.0075,  0.0742, -0.0174,  0.0591, -0.0048,  0.0569,
         0.0030,  0.0572,

___________
hamiltonian
[tensor([-0.0010,  0.0874, -0.0034,  0.0835, -0.0066,  0.0790, -0.0109,  0.0722,
        -0.0144,  0.0634, -0.0175,  0.0549, -0.0196,  0.0454, -0.0213,  0.0361,
        -0.0225,  0.0271, -0.0221,  0.0178, -0.0227,  0.0111, -0.0219,  0.0037,
        -0.0208, -0.0029, -0.0196, -0.0086, -0.0177, -0.0131, -0.0150, -0.0183,
        -0.0127, -0.0251, -0.0078, -0.0290, -0.0054, -0.0385, -0.0046, -0.0477,
         0.0051, -0.0500,  0.0135, -0.0504,  0.0098, -0.0399,  0.0093, -0.0291,
         0.0143, -0.0207,  0.0166, -0.0121,  0.0189, -0.0050,  0.0197,  0.0038,
         0.0208,  0.0124,  0.0212,  0.0212,  0.0219,  0.0319,  0.0215,  0.0429,
         0.0193,  0.0521,  0.0163,  0.0606,  0.0120,  0.0685,  0.0082,  0.0739,
         0.0037,  0.0802,  0.0008,  0.0847], grad_fn=<AddBackward0>), tensor([-3.0412e-03,  8.2108e-02,  7.0769e-04,  8.2581e-02, -9.4664e-03,
         6.9558e-02, -3.7811e-03,  6.9210e-02,  1.8987e-03,  7.0351e-02,
         7.0797e-03,  7.2199e-02, -1.49

In [7]:
# Results
%matplotlib qt5
plt.subplot(1, 3, 1)
plt.axis(aabb.get_list())
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-')
plt.plot(pos_implicit1[:, 0].numpy(), pos_implicit1[:, 1].numpy(), '.')
plt.plot(pos_implicit0[:, 0].numpy(), pos_implicit0[:, 1].numpy(), 'x')

plt.subplot(1, 3, 2)
plt.axis(aabb.get_list())
out = model.shot_manifold[0].gd.view(-1, 2).detach().numpy()
shot_implicit0 = model.shot_manifold[1].gd.view(-1, 2).detach().numpy()
shot_implicit00 = model.shot_manifold[2].gd.view(-1, 2).detach().numpy()
shot_implicit1 = model.shot_manifold[3].gd[0].view(-1, 2).detach().numpy()
plt.plot(out[:, 0], out[:, 1], '-')
plt.plot(shot_implicit0[:, 0], shot_implicit0[:, 1], 'x')
plt.plot(shot_implicit00[:, 0], shot_implicit00[:, 1], 'o')
plt.plot(shot_implicit1[:, 0], shot_implicit1[:, 1], '.')

plt.subplot(1, 3, 3)
plt.axis(aabb.get_list())
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')
plt.plot(out[:, 0], out[:, 1], '-')
plt.show()

In [8]:
# Now we only shoot with the implicit module of order 1 and the previously learned controls

compound = dm.deformationmodules.CompoundModule(model.modules)
compound.manifold.fill(model.init_manifold.copy())

itt = 10
out, controls = dm.shooting.shoot_euler(dm.hamiltonian.Hamiltonian(compound), itt)

compound2 = dm.deformationmodules.CompoundModule([model.modules[0], model.modules[1]])
#compound2.manifold.fill_gd([model.init_manifold[0].gd.clone(), model.init_manifold[1].gd.clone(), model.init_manifold[2].gd.clone(), (model.init_manifold[3].gd[0].clone(), model.init_manifold[3].gd[1].clone())])
#compound2.manifold.fill_cotan([model.init_manifold[0].cotan.clone(), model.init_manifold[1].cotan.clone(), model.init_manifold[2].cotan.clone(), (model.init_manifold[3].cotan[0].clone(), model.init_manifold[3].cotan[1].clone())])
compound2.manifold.fill_gd([model.init_manifold[0].gd.clone(), model.init_manifold[1].gd.clone()])
compound2.manifold.fill_cotan([model.init_manifold[0].cotan.clone(), model.init_manifold[1].cotan.clone()])

controls2 = [(c[0], c[1]) for c in controls]
out = dm.shooting.shoot_euler_controls(dm.hamiltonian.Hamiltonian(compound2), controls2, itt)

plt.axis('equal')
plt.plot(pos_source[:, 0].numpy(), pos_source[:, 1].numpy(), '-', color='red')
plt.plot(compound2[0].manifold.gd.view(-1, 2)[:, 0].detach().numpy(), compound2[0].manifold.gd.view(-1, 2)[:, 1].detach().numpy(), '-')
plt.plot(compound2[1].manifold.gd.view(-1, 2)[:, 0].detach().numpy(), compound2[1].manifold.gd.view(-1, 2)[:, 1].detach().numpy(), '.')
plt.plot(pos_target[:, 0].numpy(), pos_target[:, 1].numpy(), '-')
plt.show()

NameError: name 'intermediate' is not defined

In [9]:
# Evolution of the cost with iterations
plt.title("Cost")
plt.xlabel("Iteration(s)")
plt.ylabel("Cost")
plt.plot(range(len(costs)), costs)
plt.show()